In [1]:
from gapfinder.convert import convert_tif_to_png
import os

# convert the files from the tif format to the png format

The images should be placed into folders like the examples

In [2]:
input_folder = "./images/tif_images"
output_folder = "./images/raw_images"

In [ ]:
# Extract metadata from folder structure
# images\tif_images\Wild Type - 500uE 1 hour\2022-1-10\Block 1\13 WT-500uE 29k.tif
# genotype_condition would be: ['Wild Type', '500uE 1 hour']
# genotype would be: 'Wild Type'
# condition would be: '500uE 1 hour'
# date would be: '2022-1-10'
# block would be: '1'
# layer_number would be: '13'    
# k_value would be: '29'
    
convert_tif_to_png(input_folder, output_folder, csv_filename='./metadata/image_index.csv')

# select the grana locations of interest

Select the grana centroids by running the next block. It will create a popup, and allow you to cycle through the images and select the grana centroids. The data will be saved in the `centroids.csv` file in the input image folder.

Available commands:
* n (or tab, or t): Move to the next image, saving the ROI list for this image.
* r (or e): Reset the current image and remove the ROIs.
* q: Quit the program and save the ROI data processed for all images so far.

In [ ]:
from gapfinder.grana_selection import process_images
import os

input_folder = './images/raw_images'
output_filename = 'centroids.csv'
verbose = True
append = False

# delete the existing centroids? Or append to them?
if (append == False):
    if os.path.exists(output_filename):
        os.remove(output_filename)

process_images(input_folder, output_filename, verbose)

# Use the centroids to extract a region of interest around that point

The extract_strips_from_images takes the centroids and extracts a region of interest around that point. The region of interest is a square of size `size` centered at the centroid. The extracted images are saved in the `rough_rois` folder in the input image folder. If the region of interest would go outside the image, the image is skipped.

In [ ]:
from gapfinder.strip_extraction import extract_strips_from_images

raw_images_folder = './images/raw_images'
centroids_csv = f"{raw_images_folder}/centroids.csv"
rough_rois_images_folder = './images/rough_roi_images'

extract_strips_from_images(centroids_csv, raw_images_folder, rough_rois_images_folder, size=200)

# Rotate the roi images to align the grana

This requires a human's input. The images are displayed, and the user can rotate the image to align the grana. The rotated images are saved in the `rotated` folder in the input image folder, and will be used later by the scripts.

Available commands:
* s: Save the current rotation and move to the next image.
* r: Reset the current image and remove the line.
* q: Quit the program and save the rotation angles processed so far.
* h: Display this help message again.

In [ ]:
from gapfinder.strip_rotation import rotate_strips
import os

# Call the function with the desired arguments
input_folder = './images/rough_roi_images'
output_folder = './images/rotated_images'
rotation_file = 'rotation_angles.txt'
append = False

if (append == False):
    existing_data_path = os.path.join(output_folder, rotation_file)
    if os.path.exists(existing_data_path):
        os.remove(existing_data_path)

rotate_strips(input_folder, output_folder, rotation_file)

# After rotation, apply that rotation to the original images and extract context regions

This function will iterate through the centers of the rough rois and rotation angles that you created, and re-extract the regions of interest from the original images. Only centroids that have a rotation angle will be processed. The extracted images are saved in the `rotated_images` folder in the image folder.

The steps are:

1. Load the original image.
2. Shift the image so the grana of choice is centered in the image.
3. Rotate the image by the rotation angle around the center.
4. Extract a region of interest around the grana, using a box. This will be padded with the `box_size` parameter.
5. Save the extracted images in `./images/rotated_images` to be used later, overwriting the previous images.


In [ ]:
from gapfinder.context_extraction import extract_rotated_context_image, collate_centroids_and_rotation


input_folder = './images/raw_images'
output_folder = './images/rotated_images'
rough_rois_csv = "./images/rough_roi_images/strip_metadata.csv"
rotation_file = f"{output_folder}/rotation_angles.txt"
output_csv = f"{output_folder}/context_metadata.csv"

collated_metadata = collate_centroids_and_rotation(rough_rois_csv, rotation_file)

extract_rotated_context_image(input_folder, output_folder, collated_metadata, output_csv, box_size=600)

# Draw the boundaries of the ROI images

This requires a human's input. The images are displayed, and the user can draw the boundaries of the grana. The images are saved in the `boundaries` folder in the input image folder, and will be used later by the scripts. 

Notes:
1. Allow some stroma to be included in the upper and lower boundary. This aids the find_peaks algorithm.
2. If you provide an 'roi_width' parameter greater than 0, the script will force the described width for the ROI. This is useful if want to set a fixed width for the ROI, instead of drawing it manually.

Available commands:
* s: Save the current boundary and move to the next image.
* a: Rotate the image counterclockwise by 0.5 degrees.
* d: Rotate the image clockwise by 0.5 degree.
* n: Move to the next image, without saving the current boundary.
* r: Reset the current image to the original image, removing the boundary and the rotation.
* q: Quit the program and save the boundaries processed so far, exporting the sliced roi images.

In [1]:
from gapfinder.roi_boundaries import select_rois
import os

input_folder = './images/rotated_images'
output_folder = './images/roi_images'
roi_metadata_csv = f"{output_folder}/roi_metadata.csv"
metadata_file = f"{input_folder}/context_metadata.csv"
roi_width = 60

append = True

if (append == False):
    if os.path.exists(roi_metadata_csv):
        os.remove(roi_metadata_csv)

select_rois(input_folder, output_folder, roi_metadata_csv, metadata_file, roi_width=roi_width)

Select ROI for strip_0.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_1.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_10.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_11.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_12.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_13.png. Press 's' to save ROI, 'q' to quit.
Skipping strip_13.png.
Select ROI for strip_14.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_15.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_16.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_17.png. Press 's' to save ROI, 'q' to quit.
Skipping strip_17.png.
Select ROI for strip_18.png. Press 's' to save ROI, 'q' to quit.
Skipping strip_18.png.
Select ROI for strip_19.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_2.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_20.png. Press 's' to save ROI, 'q' to quit.
Select ROI for strip_21.